In [1]:
import os
import pathlib

import findspark

prj_dir = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_dir / 'spark-3.5.0-bin-hadoop3')

findspark.init(spark_home)

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext


conf = SparkConf().setMaster('local[2]').setAppName('Join Operation').set('spark.executer.memory', '2g')
sc = SparkContext(conf=conf)
ssc = StreamingContext(sparkContext=sc, batchDuration=5)

In [3]:
queued_rdds = []

for i in range(5):
    rdd = ssc.sparkContext.parallelize(range(1000))
    queued_rdds.append(rdd)


In [4]:
queue_stream = ssc.queueStream(queued_rdds)

In [6]:
ten_remainders = queue_stream.map(lambda x: (x % 10, 1)).reduceByKey(lambda x, y: x + y)
five_remainders = queue_stream.map(lambda x: (x % 5, 1)).reduceByKey(lambda x, y: x + y)

ten_remainders.join(five_remainders).pprint()

In [7]:
ssc.start()

-------------------------------------------
Time: 2023-11-20 14:27:45
-------------------------------------------
(0, (100, 200))
(2, (100, 200))
(4, (100, 200))
(1, (100, 200))
(3, (100, 200))

-------------------------------------------
Time: 2023-11-20 14:27:45
-------------------------------------------
(0, (100, 200))
(2, (100, 200))
(4, (100, 200))
(1, (100, 200))
(3, (100, 200))



In [ ]:
ssc.stop()